# Graft Loss: Three Dataset Parallel Execution Driver

**Enhanced Pipeline Management for AWS Linux 2023 EC2**

This notebook provides comprehensive management and monitoring for running the graft-loss pipeline across three datasets in parallel:

- **Original Study** (2010-2019) 
- **Full Dataset with COVID** (2010-2023)
- **Full Dataset without COVID** (2010-2019, excluding 2020+)

## Key Features:
- 🚀 Parallel execution with resource optimization
- 📊 Real-time monitoring with progress logging
- 💾 Memory optimization for large datasets (1TB RAM support)
- 🎯 Background execution with PID tracking
- 🔒 Idempotent design - safe to run multiple times

All logs and step summaries are written to `logs/` by enhanced pipeline logger. Use the cells below to execute runs, then review summaries and monitoring outputs.

In [1]:
# Enhanced Resource Monitor with Memory Management
# Prevents browser crashes during monitoring

cat("🖥️  System Resource Monitor\n")
cat("========================\n")
flush.console()

# Memory management: Clear environment
rm(list = ls())
gc(verbose = FALSE)

# Check system resources
memory_info <- function() {
  tryCatch({
    if (file.exists("/proc/meminfo")) {
      meminfo <- readLines("/proc/meminfo")
      total <- as.numeric(gsub(".*: *([0-9]+) kB", "\\1", meminfo[grep("MemTotal", meminfo)])) / 1024 / 1024
      avail <- as.numeric(gsub(".*: *([0-9]+) kB", "\\1", meminfo[grep("MemAvailable", meminfo)])) / 1024 / 1024  
      sprintf("Memory: %.1f GB available of %.1f GB total", avail, total)
    } else {
      "Memory info not available"
    }
  }, error = function(e) "Memory detection failed")
}

cat(memory_info(), "\n")
flush.console()

SyntaxError: invalid syntax (1612433403.py, line 13)

## ⚙️ EC2 Troubleshooting

If experiencing file load errors on EC2, run the diagnostic cell below first.

In [ ]:
# EC2 File Load Error Diagnostics & Fix
# Run this first if experiencing file loading issues on EC2

cat("🔧 EC2 Diagnostics & Startup Check\n")
cat("==================================\n")

# Check critical files exist
critical_files <- c(
  "scripts/run_three_datasets.R",
  "scripts/enhanced_pipeline_logger_v2.R", 
  "R/utils/parallel_utils.R",
  "scripts/run_pipeline.R"
)

all_ok <- TRUE
for (file in critical_files) {
  if (file.exists(file)) {
    cat(sprintf("✅ %s\n", file))
  } else {
    cat(sprintf("❌ %s (MISSING)\n", file))
    all_ok <- FALSE
  }
}

# Check directories
required_dirs <- c("logs", "R/utils", "scripts")
for (dir in required_dirs) {
  if (!dir.exists(dir)) {
    dir.create(dir, recursive = TRUE, showWarnings = FALSE)
    cat(sprintf("✅ Created directory: %s\n", dir))
  }
}

# Set environment for EC2
Sys.setenv(R_MAX_VSIZE = "950Gb")
Sys.setenv(OMP_NUM_THREADS = "1")
cat("✅ Environment configured for EC2\n")

if (all_ok) {
  cat("\n🟢 All systems ready for pipeline execution!\n")
} else {
  cat("\n🔴 Issues detected - fix missing files before proceeding\n")
}

flush.console()

## 🚀 Launch Parallel Execution

The launcher below is **idempotent** - safe to run multiple times. It will detect existing processes and prevent duplicate launches.

In [ ]:
# Idempotent 3-Dataset Pipeline Launcher
# Ensures only one instance runs - safe to execute multiple times

dir.create("logs", showWarnings = FALSE)

# Set R memory environment
Sys.setenv(R_MAX_VSIZE = "950Gb")

# Check if orchestrator exists
if (!file.exists("scripts/run_three_datasets.R")) {
  cat("❌ Error: scripts/run_three_datasets.R not found\n")
  cat("Available scripts:\n")
  print(list.files("scripts", pattern = "\\.R$"))
  stop("Missing orchestrator script")
}

# Idempotent check: Look for running pipeline processes
check_running_pipelines <- function() {
  # Check for orchestrator process
  orch_check <- system("pgrep -f 'run_three_datasets.R'", ignore.stdout = TRUE, ignore.stderr = TRUE)
  
  # Check for individual dataset processes  
  dataset_check <- system("pgrep -f 'enhanced_pipeline_logger_v2.R'", ignore.stdout = TRUE, ignore.stderr = TRUE)
  
  # Check PID file if it exists
  pid_file_exists <- file.exists("logs/parallel_config.log")
  
  return(list(
    orchestrator_running = (orch_check == 0),
    datasets_running = (dataset_check == 0), 
    pid_file_exists = pid_file_exists
  ))
}

# Get current status
status <- check_running_pipelines()

cat("🔍 Pipeline Status Check\n")
cat("========================\n")
cat(sprintf("Orchestrator running: %s\n", ifelse(status$orchestrator_running, "✅ YES", "❌ NO")))
cat(sprintf("Dataset processes: %s\n", ifelse(status$datasets_running, "✅ ACTIVE", "❌ NONE")))
cat(sprintf("Config log exists: %s\n", ifelse(status$pid_file_exists, "✅ YES", "❌ NO")))
flush.console()

if (status$orchestrator_running || status$datasets_running) {
  cat("\n🟡 PIPELINE ALREADY RUNNING\n")
  cat("=========================\n")
  cat("The 3-dataset parallel pipeline is already active.\n")
  cat("Use the monitoring cells below to check progress.\n")
  cat("To restart, first stop the current run in the management cell.\n\n")
  flush.console()
  
  # Show current PIDs if available
  if (status$pid_file_exists) {
    cat("📋 Current process info:\n")
    tryCatch({
      recent_log <- tail(readLines("logs/parallel_config.log"), 3)
      cat(paste(recent_log, collapse = "\n"), "\n")
    }, error = function(e) cat("Could not read recent log entries\n"))
  }
  
} else {
  cat("\n🚀 LAUNCHING NEW PIPELINE\n")
  cat("=========================\n")
  flush.console()
  
  # Clean up old logs to start fresh
  old_logs <- list.files("logs", pattern = "(orch_|parallel_config)", full.names = TRUE)
  if (length(old_logs) > 0) {
    file.remove(old_logs)
    cat("🧹 Cleared old log files\n")
  }
  
  # Launch the pipeline
  launch_cmd <- 'nohup Rscript scripts/run_three_datasets.R > logs/orch_parallel.log 2>&1 & echo $!'
  
  result <- tryCatch({
    pid_output <- system(launch_cmd, intern = TRUE)
    pid_output
  }, error = function(e) {
    paste("ERROR:", e$message)
  })
  
  # Process result
  if (length(result) > 0 && !grepl("ERROR|error|failed", result[1], ignore.case = TRUE)) {
    pid <- result[1]
    
    if (grepl("^[0-9]+$", pid)) {
      cat("✅ Successfully launched parallel orchestrator!\n")
      cat(sprintf("Process ID: %s\n", pid))
      cat("System: AWS Linux 2023 EC2 (1TB RAM)\n")
      cat("Memory: R_MAX_VSIZE=950Gb\n")
      cat("Orchestrator log: logs/orch_parallel.log\n\n")
      flush.console()
      
      cat("📊 Expected dataset logs:\n")
      cat("  - logs/orch_bg_original_study.log\n")
      cat("  - logs/orch_bg_full_with_covid.log\n") 
      cat("  - logs/orch_bg_full_without_covid.log\n\n")
      flush.console()
      
      # Log the successful launch with timestamp
      launch_log <- sprintf("Pipeline launched: PID %s at %s\n", pid, Sys.time())
      cat(launch_log, file = "logs/parallel_config.log", append = TRUE)
      
      cat("🔍 Use monitoring cells below to track progress\n")
      flush.console()
      
    } else {
      cat("⚠️  Launch completed but PID format unexpected:", pid, "\n")
      flush.console()
    }
  } else {
    cat("❌ Launch failed\n")
    cat("Output:", paste(result, collapse = "\n"), "\n")
    flush.console()
  }
}

In [ ]:
# Pipeline Management - Stop/Restart Control
# Idempotent management of pipeline processes

cat("🎛️  Pipeline Management Control\n")
cat("===============================\n")

# Function to safely stop all pipeline processes
stop_pipeline <- function() {
  stopped_count <- 0
  
  # Stop orchestrator processes
  orch_pids <- system("pgrep -f 'run_three_datasets.R'", intern = TRUE, ignore.stderr = TRUE)
  if (length(orch_pids) > 0 && !any(grepl("ERROR", orch_pids))) {
    for (pid in orch_pids) {
      if (grepl("^[0-9]+$", pid)) {
        system(sprintf("kill %s", pid), ignore.stdout = TRUE, ignore.stderr = TRUE)
        stopped_count <- stopped_count + 1
        cat(sprintf("🛑 Stopped orchestrator PID %s\n", pid))
      }
    }
  }
  
  # Stop dataset logger processes  
  logger_pids <- system("pgrep -f 'enhanced_pipeline_logger_v2.R'", intern = TRUE, ignore.stderr = TRUE)
  if (length(logger_pids) > 0 && !any(grepl("ERROR", logger_pids))) {
    for (pid in logger_pids) {
      if (grepl("^[0-9]+$", pid)) {
        system(sprintf("kill %s", pid), ignore.stdout = TRUE, ignore.stderr = TRUE)
        stopped_count <- stopped_count + 1
        cat(sprintf("🛑 Stopped dataset logger PID %s\n", pid))
      }
    }
  }
  
  # Stop any remaining R processes running the pipeline
  pipeline_pids <- system("pgrep -f 'run_pipeline.R'", intern = TRUE, ignore.stderr = TRUE)
  if (length(pipeline_pids) > 0 && !any(grepl("ERROR", pipeline_pids))) {
    for (pid in pipeline_pids) {
      if (grepl("^[0-9]+$", pid)) {
        system(sprintf("kill %s", pid), ignore.stdout = TRUE, ignore.stderr = TRUE)  
        stopped_count <- stopped_count + 1
        cat(sprintf("🛑 Stopped pipeline process PID %s\n", pid))
      }
    }
  }
  
  return(stopped_count)
}

# Check current status first
orch_running <- system("pgrep -f 'run_three_datasets.R'", ignore.stdout = TRUE, ignore.stderr = TRUE) == 0
datasets_running <- system("pgrep -f 'enhanced_pipeline_logger_v2.R'", ignore.stdout = TRUE, ignore.stderr = TRUE) == 0

cat(sprintf("Current status: %s\n", 
    ifelse(orch_running || datasets_running, "🟢 RUNNING", "🔴 STOPPED")))

if (orch_running || datasets_running) {
  cat("\n⚠️  STOPPING ALL PIPELINE PROCESSES\n")
  cat("This will gracefully terminate the current run.\n")
  flush.console()
  
  stopped <- stop_pipeline()
  
  if (stopped > 0) {
    cat(sprintf("\n✅ Successfully stopped %d processes\n", stopped))
    cat("Pipeline is now stopped. You can safely restart using the launch cell.\n")
    
    # Log the stop action
    stop_log <- sprintf("Pipeline stopped: %d processes terminated at %s\n", stopped, Sys.time())
    cat(stop_log, file = "logs/parallel_config.log", append = TRUE)
  } else {
    cat("\n🤷 No pipeline processes found to stop\n")
  }
} else {
  cat("\n✅ No pipeline processes are currently running\n")
  cat("You can start the pipeline using the launch cell above.\n")
}

# Clean memory after management operations
gc(verbose = FALSE)
flush.console()

In [ ]:
# Real-time Pipeline Status Monitor
# Safe to run multiple times - shows current state

cat("📊 Pipeline Status Monitor\n")
cat("==========================\n")

# Memory management for browser safety
gc(verbose = FALSE)

# Check process status
check_pipeline_status <- function() {
  # Get orchestrator PIDs
  orch_pids <- tryCatch({
    pids <- system("pgrep -f 'run_three_datasets.R'", intern = TRUE, ignore.stderr = TRUE)
    if (length(pids) > 0 && !any(grepl("ERROR", pids)) && all(grepl("^[0-9]+$", pids))) {
      pids
    } else {
      character(0)
    }
  }, error = function(e) character(0))
  
  # Get dataset logger PIDs  
  logger_pids <- tryCatch({
    pids <- system("pgrep -f 'enhanced_pipeline_logger_v2.R'", intern = TRUE, ignore.stderr = TRUE)
    if (length(pids) > 0 && !any(grepl("ERROR", pids)) && all(grepl("^[0-9]+$", pids))) {
      pids
    } else {
      character(0)
    }
  }, error = function(e) character(0))
  
  return(list(
    orchestrator = orch_pids,
    loggers = logger_pids,
    total_processes = length(orch_pids) + length(logger_pids)
  ))
}

# Get current status
status <- check_pipeline_status()

# Display status with timestamp
cat(sprintf("Status at %s\n", format(Sys.time(), "%Y-%m-%d %H:%M:%S")))
cat("─────────────────────────────\n")

if (status$total_processes == 0) {
  cat("🔴 PIPELINE STOPPED\n")
  cat("No active processes found.\n")
} else {
  cat("🟢 PIPELINE ACTIVE\n")
  cat(sprintf("Total processes: %d\n", status$total_processes))
  
  if (length(status$orchestrator) > 0) {
    cat(sprintf("Orchestrator PIDs: %s\n", paste(status$orchestrator, collapse = ", ")))
  }
  
  if (length(status$loggers) > 0) {
    cat(sprintf("Dataset loggers: %d processes\n", length(status$loggers)))
  }
}

# Check log files and show recent activity
cat("\n📋 Log File Status\n")
cat("─────────────────\n")

log_files <- c(
  "logs/orch_parallel.log",
  "logs/orch_bg_original_study.log", 
  "logs/orch_bg_full_with_covid.log",
  "logs/orch_bg_full_without_covid.log"
)

for (log_file in log_files) {
  if (file.exists(log_file)) {
    info <- file.info(log_file)
    age_min <- as.numeric(difftime(Sys.time(), info$mtime, units = "mins"))
    cat(sprintf("✅ %s (%.1f MB, %.0f min ago)\n", 
               basename(log_file), info$size/1024/1024, age_min))
  } else {
    cat(sprintf("❌ %s (not found)\n", basename(log_file)))
  }
}

# Show system resources if available
cat("\n💻 System Resources\n")
cat("──────────────────\n")

tryCatch({
  if (file.exists("/proc/meminfo")) {
    meminfo <- readLines("/proc/meminfo")
    total_mem <- as.numeric(gsub(".*: *([0-9]+) kB", "\\1", meminfo[grep("MemTotal", meminfo)])) / 1024 / 1024
    avail_mem <- as.numeric(gsub(".*: *([0-9]+) kB", "\\1", meminfo[grep("MemAvailable", meminfo)])) / 1024 / 1024
    used_mem <- total_mem - avail_mem
    cat(sprintf("Memory: %.1f/%.1f GB used (%.1f%% utilization)\n", 
               used_mem, total_mem, (used_mem/total_mem)*100))
  }
  
  # Load average if available
  if (file.exists("/proc/loadavg")) {
    load_avg <- readLines("/proc/loadavg")[1]
    load_vals <- strsplit(load_avg, " ")[[1]][1:3]
    cat(sprintf("Load avg: %s (1m, 5m, 15m)\n", paste(load_vals, collapse = ", ")))
  }
}, error = function(e) {
  cat("System info not available\n")
})

# Flush output and clean memory
flush.console()
gc(verbose = FALSE)

In [ ]:
# Pipeline Progress Tracker  
# Idempotent monitoring - safe to run repeatedly

cat("📈 Pipeline Progress Tracker\n")
cat("============================\n")

# Memory cleanup for browser safety
rm(list = setdiff(ls(), c()))  # Keep only essential variables
gc(verbose = FALSE)

# Function to safely read recent log lines
safe_tail <- function(file_path, lines = 10) {
  tryCatch({
    if (file.exists(file_path) && file.size(file_path) > 0) {
      recent <- tail(readLines(file_path, warn = FALSE), lines)
      recent[nzchar(recent)]  # Remove empty lines
    } else {
      character(0)
    }
  }, error = function(e) character(0))
}

# Check if pipeline is active
pipeline_active <- function() {
  orch_running <- system("pgrep -f 'run_three_datasets.R'", ignore.stdout = TRUE, ignore.stderr = TRUE) == 0
  loggers_running <- system("pgrep -f 'enhanced_pipeline_logger_v2.R'", ignore.stdout = TRUE, ignore.stderr = TRUE) == 0
  return(orch_running || loggers_running)
}

is_active <- pipeline_active()
cat(sprintf("Pipeline Status: %s\n", ifelse(is_active, "🟢 ACTIVE", "🔴 INACTIVE")))
cat(sprintf("Check Time: %s\n\n", format(Sys.time(), "%H:%M:%S")))

if (!is_active) {
  cat("ℹ️  No active pipeline processes detected.\n")
  cat("Use the launch cell to start the pipeline.\n")
} else {
  cat("📊 Recent Activity from Dataset Logs\n")
  cat("───────────────────────────────────\n")
  
  datasets <- list(
    "Original Study (2010-2019)" = "logs/orch_bg_original_study.log",
    "Full with COVID" = "logs/orch_bg_full_with_covid.log", 
    "Full without COVID" = "logs/orch_bg_full_without_covid.log"
  )
  
  for (dataset_name in names(datasets)) {
    log_file <- datasets[[dataset_name]]
    recent_lines <- safe_tail(log_file, 3)
    
    cat(sprintf("\n🔸 %s:\n", dataset_name))
    if (length(recent_lines) > 0) {
      # Show recent progress with clean formatting
      for (line in recent_lines) {
        # Extract useful info and clean up formatting
        clean_line <- gsub("^\\s*\\[.*?\\]\\s*", "", line)  # Remove timestamps
        clean_line <- gsub("^\\s*", "   ", clean_line)       # Indent
        if (nchar(clean_line) > 80) {
          clean_line <- paste0(substr(clean_line, 1, 77), "...")
        }
        if (nzchar(clean_line) && !grepl("^\\s*$", clean_line)) {
          cat(clean_line, "\n")
        }
      }
    } else {
      cat("   (No recent activity or log not found)\n")
    }
  }
  
  # Check orchestrator summary
  cat(sprintf("\n📋 Orchestrator Status:\n"))
  orch_lines <- safe_tail("logs/orch_parallel.log", 5)
  if (length(orch_lines) > 0) {
    for (line in tail(orch_lines, 2)) {  # Show last 2 lines only
      clean_line <- gsub("^\\s*", "   ", line)
      if (nzchar(clean_line)) {
        cat(clean_line, "\n")
      }
    }
  } else {
    cat("   (Orchestrator log not found)\n")
  }
}

# Memory management and output flush
flush.console()
gc(verbose = FALSE)

## 🔧 System Analysis

Complete system diagnostics and resource analysis.

In [ ]:
# System Analysis & Resource Overview
# Idempotent system diagnostics - safe to run anytime

cat("🔧 System Analysis & Resource Overview\n")
cat("======================================\n")

# Memory management
gc(verbose = FALSE)

# System information gathering
system_info <- function() {
  info <- list()
  
  # Get CPU information
  tryCatch({
    if (file.exists("/proc/cpuinfo")) {
      cpuinfo <- readLines("/proc/cpuinfo")
      cpu_model <- cpuinfo[grep("model name", cpuinfo)[1]]
      if (length(cpu_model) > 0) {
        info$cpu <- gsub(".*: ", "", cpu_model)
      }
      
      core_count <- length(grep("^processor", cpuinfo))
      info$cores <- core_count
    } else {
      # Fallback for non-Linux systems
      info$cores <- as.numeric(Sys.getenv("NUMBER_OF_PROCESSORS", "unknown"))
      info$cpu <- "CPU information not available"
    }
  }, error = function(e) {
    info$cores <- "unknown"
    info$cpu <- "CPU detection failed"
  })
  
  # Memory information
  tryCatch({
    if (file.exists("/proc/meminfo")) {
      meminfo <- readLines("/proc/meminfo")
      total_mem <- as.numeric(gsub(".*: *([0-9]+) kB", "\\1", meminfo[grep("MemTotal", meminfo)])) / 1024 / 1024
      avail_mem <- as.numeric(gsub(".*: *([0-9]+) kB", "\\1", meminfo[grep("MemAvailable", meminfo)])) / 1024 / 1024
      free_mem <- as.numeric(gsub(".*: *([0-9]+) kB", "\\1", meminfo[grep("MemFree", meminfo)])) / 1024 / 1024
      cached_mem <- as.numeric(gsub(".*: *([0-9]+) kB", "\\1", meminfo[grep("^Cached", meminfo)])) / 1024 / 1024
      
      info$memory <- list(
        total = total_mem,
        available = avail_mem,
        free = free_mem,
        cached = cached_mem,
        used = total_mem - avail_mem
      )
    }
  }, error = function(e) {
    info$memory <- "Memory info not available"
  })
  
  # Disk space for current directory
  tryCatch({
    disk_info <- system("df -h . | tail -1", intern = TRUE)
    if (length(disk_info) > 0) {
      info$disk <- disk_info
    }
  }, error = function(e) {
    info$disk <- "Disk info not available"
  })
  
  return(info)
}

# Get system info
sys_info <- system_info()

# Display system configuration
cat("💻 System Configuration\n")
cat("─────────────────────\n")
cat(sprintf("CPU: %s\n", sys_info$cpu))
cat(sprintf("Cores: %s\n", sys_info$cores))

if (is.list(sys_info$memory)) {
  mem <- sys_info$memory
  cat(sprintf("Memory: %.1f GB total, %.1f GB available, %.1f GB used (%.1f%% util)\n", 
             mem$total, mem$available, mem$used, (mem$used/mem$total)*100))
  cat(sprintf("Cache: %.1f GB, Free: %.1f GB\n", mem$cached, mem$free))
} else {
  cat(sprintf("Memory: %s\n", sys_info$memory))
}

if (!is.null(sys_info$disk)) {
  cat(sprintf("Disk: %s\n", sys_info$disk))
}

# R Environment
cat(sprintf("R Version: %s\n", R.version.string))
cat(sprintf("R_MAX_VSIZE: %s\n", Sys.getenv("R_MAX_VSIZE", "not set")))

# Pipeline-specific analysis  
cat("\n🏗️ Pipeline Environment\n")
cat("────────────────────\n")

# Check core allocation for pipeline
if (is.numeric(sys_info$cores) && sys_info$cores > 0) {
  cores_per_dataset <- floor(sys_info$cores * 0.25)
  total_pipeline_cores <- cores_per_dataset * 3
  cat(sprintf("Pipeline cores per dataset: %d (25%% of %d)\n", cores_per_dataset, sys_info$cores))
  cat(sprintf("Total pipeline utilization: %d cores (%.1f%%)\n", 
             total_pipeline_cores, (total_pipeline_cores/sys_info$cores)*100))
}

# Check critical directories and files
cat("\n📁 Project Structure\n")
cat("──────────────────\n")

critical_paths <- c(
  "scripts/run_three_datasets.R",
  "scripts/enhanced_pipeline_logger_v2.R", 
  "R/utils/parallel_utils.R",
  "logs/"
)

for (path in critical_paths) {
  if (file.exists(path)) {
    if (dir.exists(path)) {
      files_count <- length(list.files(path))
      cat(sprintf("✅ %s (%d files)\n", path, files_count))
    } else {
      info <- file.info(path)
      cat(sprintf("✅ %s (%.1f KB)\n", path, info$size/1024))
    }
  } else {
    cat(sprintf("❌ %s (missing)\n", path))
  }
}

# Current pipeline status summary
cat(sprintf("\n⏰ Analysis completed at %s\n", format(Sys.time(), "%Y-%m-%d %H:%M:%S")))

# Clean up and flush
flush.console()
gc(verbose = FALSE)

# Graft Loss: Three Dataset Parallel Execution Driver

**Enhanced Pipeline Management for AWS Linux 2023 EC2**

This notebook provides comprehensive management and monitoring for running the graft-loss pipeline across three datasets in parallel:

- **Original Study** (2010-2019) 
- **Full Dataset with COVID** (2010-2023)
- **Full Dataset without COVID** (2010-2019, excluding 2020+)

## Key Features:
- 🚀 Parallel execution with resource optimization
- 📊 Real-time monitoring with progress logging
- 💾 Memory optimization for large datasets (1TB RAM support)
- 🎯 Background execution with PID tracking

All logs and step summaries are written to `logs/` by enhanced pipeline logger. Use the cells below to execute runs, then review summaries and monitoring outputs.

In [ ]:
# Enhanced Resource Monitor with Memory Management
# Prevents browser crashes during monitoring

cat("🖥️  System Resource Monitor\n")
cat("========================\n")
flush.console()

# Memory management: Clear environment
rm(list = ls())
gc(verbose = FALSE)

# Check system resources
memory_info <- function() {
  tryCatch({
    if (file.exists("/proc/meminfo")) {
      meminfo <- readLines("/proc/meminfo")
      total <- as.numeric(gsub(".*: *([0-9]+) kB", "\\1", meminfo[grep("MemTotal", meminfo)])) / 1024 / 1024
      avail <- as.numeric(gsub(".*: *([0-9]+) kB", "\\1", meminfo[grep("MemAvailable", meminfo)])) / 1024 / 1024  
      sprintf("Memory: %.1f GB available of %.1f GB total", avail, total)
    } else {
      "Memory info not available"
    }
  }, error = function(e) "Memory detection failed")
}

cat(memory_info(), "\n")
flush.console()

## 🚀 Launch Parallel Execution

This section launches all three datasets in parallel using the enhanced pipeline logger with resource monitoring.

**Expected Execution Flow:**
- **Original Study** (2010-2019) → `logs/orch_bg_original_study.log`
- **Full with COVID** (2010-2023) → `logs/orch_bg_full_with_covid.log`  
- **Full without COVID** (2010-2019, excl. 2020+) → `logs/orch_bg_full_without_covid.log`

**Parallel Configuration:**
- Each dataset gets ~25% of available cores for stable execution
- Forces inner model threads to 1 to avoid oversubscription
- **3 separate PIDs** for true parallel execution

## ⚙️ EC2 Troubleshooting

If experiencing file load errors on EC2, run the diagnostic tools below first.

In [ ]:
# Idempotent 3-Dataset Pipeline Launcher
# Ensures only one instance runs - safe to execute multiple times

dir.create("logs", showWarnings = FALSE)

# Set R memory environment
Sys.setenv(R_MAX_VSIZE = "950Gb")

# Check if orchestrator exists
if (!file.exists("scripts/run_three_datasets.R")) {
  cat("❌ Error: scripts/run_three_datasets.R not found\n")
  cat("Available scripts:\n")
  print(list.files("scripts", pattern = "\\.R$"))
  stop("Missing orchestrator script")
}

# Idempotent check: Look for running pipeline processes
check_running_pipelines <- function() {
  # Check for orchestrator process
  orch_check <- system("pgrep -f 'run_three_datasets.R'", ignore.stdout = TRUE, ignore.stderr = TRUE)
  
  # Check for individual dataset processes  
  dataset_check <- system("pgrep -f 'enhanced_pipeline_logger_v2.R'", ignore.stdout = TRUE, ignore.stderr = TRUE)
  
  # Check PID file if it exists
  pid_file_exists <- file.exists("logs/parallel_config.log")
  
  return(list(
    orchestrator_running = (orch_check == 0),
    datasets_running = (dataset_check == 0), 
    pid_file_exists = pid_file_exists
  ))
}

# Get current status
status <- check_running_pipelines()

cat("🔍 Pipeline Status Check\n")
cat("========================\n")
cat(sprintf("Orchestrator running: %s\n", ifelse(status$orchestrator_running, "✅ YES", "❌ NO")))
cat(sprintf("Dataset processes: %s\n", ifelse(status$datasets_running, "✅ ACTIVE", "❌ NONE")))
cat(sprintf("Config log exists: %s\n", ifelse(status$pid_file_exists, "✅ YES", "❌ NO")))
flush.console()

if (status$orchestrator_running || status$datasets_running) {
  cat("\n? PIPELINE ALREADY RUNNING\n")
  cat("=========================\n")
  cat("The 3-dataset parallel pipeline is already active.\n")
  cat("Use the monitoring cells below to check progress.\n")
  cat("To restart, first stop the current run in the management cell.\n\n")
  flush.console()
  
  # Show current PIDs if available
  if (status$pid_file_exists) {
    cat("📋 Current process info:\n")
    tryCatch({
      recent_log <- tail(readLines("logs/parallel_config.log"), 3)
      cat(paste(recent_log, collapse = "\n"), "\n")
    }, error = function(e) cat("Could not read recent log entries\n"))
  }
  
} else {
  cat("\n🚀 LAUNCHING NEW PIPELINE\n")
  cat("=========================\n")
  flush.console()
  
  # Clean up old logs to start fresh
  old_logs <- list.files("logs", pattern = "(orch_|parallel_config)", full.names = TRUE)
  if (length(old_logs) > 0) {
    file.remove(old_logs)
    cat("🧹 Cleared old log files\n")
  }
  
  # Launch the pipeline
  launch_cmd <- 'nohup Rscript scripts/run_three_datasets.R > logs/orch_parallel.log 2>&1 & echo $!'
  
  result <- tryCatch({
    pid_output <- system(launch_cmd, intern = TRUE)
    pid_output
  }, error = function(e) {
    paste("ERROR:", e$message)
  })
  
  # Process result
  if (length(result) > 0 && !grepl("ERROR|error|failed", result[1], ignore.case = TRUE)) {
    pid <- result[1]
    
    if (grepl("^[0-9]+$", pid)) {
      cat("✅ Successfully launched parallel orchestrator!\n")
      cat(sprintf("Process ID: %s\n", pid))
      cat("System: AWS Linux 2023 EC2 (1TB RAM)\n")
      cat("Memory: R_MAX_VSIZE=950Gb\n")
      cat("Orchestrator log: logs/orch_parallel.log\n\n")
      flush.console()
      
      cat("📊 Expected dataset logs:\n")
      cat("  - logs/orch_bg_original_study.log\n")
      cat("  - logs/orch_bg_full_with_covid.log\n") 
      cat("  - logs/orch_bg_full_without_covid.log\n\n")
      flush.console()
      
      # Log the successful launch with timestamp
      launch_log <- sprintf("Pipeline launched: PID %s at %s\n", pid, Sys.time())
      cat(launch_log, file = "logs/parallel_config.log", append = TRUE)
      
      cat("🔍 Use monitoring cells below to track progress\n")
      flush.console()
      
    } else {
      cat("⚠️  Launch completed but PID format unexpected:", pid, "\n")
      flush.console()
    }
  } else {
    cat("❌ Launch failed\n")
    cat("Output:", paste(result, collapse = "\n"), "\n")
    flush.console()
  }
}

In [ ]:
# Pipeline Management - Stop/Restart Control
# Idempotent management of pipeline processes

cat("🎛️  Pipeline Management Control\n")
cat("===============================\n")

# Function to safely stop all pipeline processes
stop_pipeline <- function() {
  stopped_count <- 0
  
  # Stop orchestrator processes
  orch_pids <- system("pgrep -f 'run_three_datasets.R'", intern = TRUE, ignore.stderr = TRUE)
  if (length(orch_pids) > 0 && !any(grepl("ERROR", orch_pids))) {
    for (pid in orch_pids) {
      if (grepl("^[0-9]+$", pid)) {
        system(sprintf("kill %s", pid), ignore.stdout = TRUE, ignore.stderr = TRUE)
        stopped_count <- stopped_count + 1
        cat(sprintf("🛑 Stopped orchestrator PID %s\n", pid))
      }
    }
  }
  
  # Stop dataset logger processes  
  logger_pids <- system("pgrep -f 'enhanced_pipeline_logger_v2.R'", intern = TRUE, ignore.stderr = TRUE)
  if (length(logger_pids) > 0 && !any(grepl("ERROR", logger_pids))) {
    for (pid in logger_pids) {
      if (grepl("^[0-9]+$", pid)) {
        system(sprintf("kill %s", pid), ignore.stdout = TRUE, ignore.stderr = TRUE)
        stopped_count <- stopped_count + 1
        cat(sprintf("🛑 Stopped dataset logger PID %s\n", pid))
      }
    }
  }
  
  # Stop any remaining R processes running the pipeline
  pipeline_pids <- system("pgrep -f 'run_pipeline.R'", intern = TRUE, ignore.stderr = TRUE)
  if (length(pipeline_pids) > 0 && !any(grepl("ERROR", pipeline_pids))) {
    for (pid in pipeline_pids) {
      if (grepl("^[0-9]+$", pid)) {
        system(sprintf("kill %s", pid), ignore.stdout = TRUE, ignore.stderr = TRUE)  
        stopped_count <- stopped_count + 1
        cat(sprintf("🛑 Stopped pipeline process PID %s\n", pid))
      }
    }
  }
  
  return(stopped_count)
}

# Check current status first
orch_running <- system("pgrep -f 'run_three_datasets.R'", ignore.stdout = TRUE, ignore.stderr = TRUE) == 0
datasets_running <- system("pgrep -f 'enhanced_pipeline_logger_v2.R'", ignore.stdout = TRUE, ignore.stderr = TRUE) == 0

cat(sprintf("Current status: %s\n", 
    ifelse(orch_running || datasets_running, "🟢 RUNNING", "🔴 STOPPED")))

if (orch_running || datasets_running) {
  cat("\n⚠️  STOPPING ALL PIPELINE PROCESSES\n")
  cat("This will gracefully terminate the current run.\n")
  flush.console()
  
  stopped <- stop_pipeline()
  
  if (stopped > 0) {
    cat(sprintf("\n✅ Successfully stopped %d processes\n", stopped))
    cat("Pipeline is now stopped. You can safely restart using the launch cell.\n")
    
    # Log the stop action
    stop_log <- sprintf("Pipeline stopped: %d processes terminated at %s\n", stopped, Sys.time())
    cat(stop_log, file = "logs/parallel_config.log", append = TRUE)
  } else {
    cat("\n🤷 No pipeline processes found to stop\n")
  }
} else {
  cat("\n✅ No pipeline processes are currently running\n")
  cat("You can start the pipeline using the launch cell above.\n")
}

# Clean memory after management operations
gc(verbose = FALSE)
flush.console()

In [ ]:
# Real-time Pipeline Status Monitor
# Safe to run multiple times - shows current state

cat("? Pipeline Status Monitor\n")
cat("==========================\n")

# Memory management for browser safety
gc(verbose = FALSE)

# Check process status
check_pipeline_status <- function() {
  # Get orchestrator PIDs
  orch_pids <- tryCatch({
    pids <- system("pgrep -f 'run_three_datasets.R'", intern = TRUE, ignore.stderr = TRUE)
    if (length(pids) > 0 && !any(grepl("ERROR", pids)) && all(grepl("^[0-9]+$", pids))) {
      pids
    } else {
      character(0)
    }
  }, error = function(e) character(0))
  
  # Get dataset logger PIDs  
  logger_pids <- tryCatch({
    pids <- system("pgrep -f 'enhanced_pipeline_logger_v2.R'", intern = TRUE, ignore.stderr = TRUE)
    if (length(pids) > 0 && !any(grepl("ERROR", pids)) && all(grepl("^[0-9]+$", pids))) {
      pids
    } else {
      character(0)
    }
  }, error = function(e) character(0))
  
  return(list(
    orchestrator = orch_pids,
    loggers = logger_pids,
    total_processes = length(orch_pids) + length(logger_pids)
  ))
}

# Get current status
status <- check_pipeline_status()

# Display status with timestamp
cat(sprintf("Status at %s\n", format(Sys.time(), "%Y-%m-%d %H:%M:%S")))
cat("─────────────────────────────\n")

if (status$total_processes == 0) {
  cat("🔴 PIPELINE STOPPED\n")
  cat("No active processes found.\n")
} else {
  cat("🟢 PIPELINE ACTIVE\n")
  cat(sprintf("Total processes: %d\n", status$total_processes))
  
  if (length(status$orchestrator) > 0) {
    cat(sprintf("Orchestrator PIDs: %s\n", paste(status$orchestrator, collapse = ", ")))
  }
  
  if (length(status$loggers) > 0) {
    cat(sprintf("Dataset loggers: %d processes\n", length(status$loggers)))
  }
}

# Check log files and show recent activity
cat("\n📋 Log File Status\n")
cat("─────────────────\n")

log_files <- c(
  "logs/orch_parallel.log",
  "logs/orch_bg_original_study.log", 
  "logs/orch_bg_full_with_covid.log",
  "logs/orch_bg_full_without_covid.log"
)

for (log_file in log_files) {
  if (file.exists(log_file)) {
    info <- file.info(log_file)
    age_min <- as.numeric(difftime(Sys.time(), info$mtime, units = "mins"))
    cat(sprintf("✅ %s (%.1f MB, %.0f min ago)\n", 
               basename(log_file), info$size/1024/1024, age_min))
  } else {
    cat(sprintf("❌ %s (not found)\n", basename(log_file)))
  }
}

# Show system resources if available
cat("\n💻 System Resources\n")
cat("──────────────────\n")

tryCatch({
  if (file.exists("/proc/meminfo")) {
    meminfo <- readLines("/proc/meminfo")
    total_mem <- as.numeric(gsub(".*: *([0-9]+) kB", "\\1", meminfo[grep("MemTotal", meminfo)])) / 1024 / 1024
    avail_mem <- as.numeric(gsub(".*: *([0-9]+) kB", "\\1", meminfo[grep("MemAvailable", meminfo)])) / 1024 / 1024
    used_mem <- total_mem - avail_mem
    cat(sprintf("Memory: %.1f/%.1f GB used (%.1f%% utilization)\n", 
               used_mem, total_mem, (used_mem/total_mem)*100))
  }
  
  # Load average if available
  if (file.exists("/proc/loadavg")) {
    load_avg <- readLines("/proc/loadavg")[1]
    load_vals <- strsplit(load_avg, " ")[[1]][1:3]
    cat(sprintf("Load avg: %s (1m, 5m, 15m)\n", paste(load_vals, collapse = ", ")))
  }
}, error = function(e) {
  cat("System info not available\n")
})

# Flush output and clean memory
flush.console()
gc(verbose = FALSE)

In [ ]:
# Pipeline Progress Tracker  
# Idempotent monitoring - safe to run repeatedly

cat("📈 Pipeline Progress Tracker\n")
cat("============================\n")

# Memory cleanup for browser safety
rm(list = setdiff(ls(), c()))  # Keep only essential variables
gc(verbose = FALSE)

# Function to safely read recent log lines
safe_tail <- function(file_path, lines = 10) {
  tryCatch({
    if (file.exists(file_path) && file.size(file_path) > 0) {
      recent <- tail(readLines(file_path, warn = FALSE), lines)
      recent[nzchar(recent)]  # Remove empty lines
    } else {
      character(0)
    }
  }, error = function(e) character(0))
}

# Check if pipeline is active
pipeline_active <- function() {
  orch_running <- system("pgrep -f 'run_three_datasets.R'", ignore.stdout = TRUE, ignore.stderr = TRUE) == 0
  loggers_running <- system("pgrep -f 'enhanced_pipeline_logger_v2.R'", ignore.stdout = TRUE, ignore.stderr = TRUE) == 0
  return(orch_running || loggers_running)
}

is_active <- pipeline_active()
cat(sprintf("Pipeline Status: %s\n", ifelse(is_active, "🟢 ACTIVE", "🔴 INACTIVE")))
cat(sprintf("Check Time: %s\n\n", format(Sys.time(), "%H:%M:%S")))

if (!is_active) {
  cat("ℹ️  No active pipeline processes detected.\n")
  cat("Use the launch cell to start the pipeline.\n")
} else {
  cat("📊 Recent Activity from Dataset Logs\n")
  cat("───────────────────────────────────\n")
  
  datasets <- list(
    "Original Study (2010-2019)" = "logs/orch_bg_original_study.log",
    "Full with COVID" = "logs/orch_bg_full_with_covid.log", 
    "Full without COVID" = "logs/orch_bg_full_without_covid.log"
  )
  
  for (dataset_name in names(datasets)) {
    log_file <- datasets[[dataset_name]]
    recent_lines <- safe_tail(log_file, 3)
    
    cat(sprintf("\n? %s:\n", dataset_name))
    if (length(recent_lines) > 0) {
      # Show recent progress with clean formatting
      for (line in recent_lines) {
        # Extract useful info and clean up formatting
        clean_line <- gsub("^\\s*\\[.*?\\]\\s*", "", line)  # Remove timestamps
        clean_line <- gsub("^\\s*", "   ", clean_line)       # Indent
        if (nchar(clean_line) > 80) {
          clean_line <- paste0(substr(clean_line, 1, 77), "...")
        }
        if (nzchar(clean_line) && !grepl("^\\s*$", clean_line)) {
          cat(clean_line, "\n")
        }
      }
    } else {
      cat("   (No recent activity or log not found)\n")
    }
  }
  
  # Check orchestrator summary
  cat(sprintf("\n📋 Orchestrator Status:\n"))
  orch_lines <- safe_tail("logs/orch_parallel.log", 5)
  if (length(orch_lines) > 0) {
    for (line in tail(orch_lines, 2)) {  # Show last 2 lines only
      clean_line <- gsub("^\\s*", "   ", line)
      if (nzchar(clean_line)) {
        cat(clean_line, "\n")
      }
    }
  } else {
    cat("   (Orchestrator log not found)\n")
  }
}

# Memory management and output flush
flush.console()
gc(verbose = FALSE)

## 📊 Comprehensive System Monitoring

Advanced system monitoring and process management for long-running parallel executions.

In [ ]:
# System Analysis & Resource Overview
# Idempotent system diagnostics - safe to run anytime

cat("? System Analysis & Resource Overview\n")
cat("======================================\n")

# Memory management
gc(verbose = FALSE)

# System information gathering
system_info <- function() {
  info <- list()
  
  # Get CPU information
  tryCatch({
    if (file.exists("/proc/cpuinfo")) {
      cpuinfo <- readLines("/proc/cpuinfo")
      cpu_model <- cpuinfo[grep("model name", cpuinfo)[1]]
      if (length(cpu_model) > 0) {
        info$cpu <- gsub(".*: ", "", cpu_model)
      }
      
      core_count <- length(grep("^processor", cpuinfo))
      info$cores <- core_count
    } else {
      # Fallback for non-Linux systems
      info$cores <- as.numeric(Sys.getenv("NUMBER_OF_PROCESSORS", "unknown"))
      info$cpu <- "CPU information not available"
    }
  }, error = function(e) {
    info$cores <- "unknown"
    info$cpu <- "CPU detection failed"
  })
  
  # Memory information
  tryCatch({
    if (file.exists("/proc/meminfo")) {
      meminfo <- readLines("/proc/meminfo")
      total_mem <- as.numeric(gsub(".*: *([0-9]+) kB", "\\1", meminfo[grep("MemTotal", meminfo)])) / 1024 / 1024
      avail_mem <- as.numeric(gsub(".*: *([0-9]+) kB", "\\1", meminfo[grep("MemAvailable", meminfo)])) / 1024 / 1024
      free_mem <- as.numeric(gsub(".*: *([0-9]+) kB", "\\1", meminfo[grep("MemFree", meminfo)])) / 1024 / 1024
      cached_mem <- as.numeric(gsub(".*: *([0-9]+) kB", "\\1", meminfo[grep("^Cached", meminfo)])) / 1024 / 1024
      
      info$memory <- list(
        total = total_mem,
        available = avail_mem,
        free = free_mem,
        cached = cached_mem,
        used = total_mem - avail_mem
      )
    }
  }, error = function(e) {
    info$memory <- "Memory info not available"
  })
  
  # Disk space for current directory
  tryCatch({
    disk_info <- system("df -h . | tail -1", intern = TRUE)
    if (length(disk_info) > 0) {
      info$disk <- disk_info
    }
  }, error = function(e) {
    info$disk <- "Disk info not available"
  })
  
  return(info)
}

# Get system info
sys_info <- system_info()

# Display system configuration
cat("💻 System Configuration\n")
cat("─────────────────────\n")
cat(sprintf("CPU: %s\n", sys_info$cpu))
cat(sprintf("Cores: %s\n", sys_info$cores))

if (is.list(sys_info$memory)) {
  mem <- sys_info$memory
  cat(sprintf("Memory: %.1f GB total, %.1f GB available, %.1f GB used (%.1f%% util)\n", 
             mem$total, mem$available, mem$used, (mem$used/mem$total)*100))
  cat(sprintf("Cache: %.1f GB, Free: %.1f GB\n", mem$cached, mem$free))
} else {
  cat(sprintf("Memory: %s\n", sys_info$memory))
}

if (!is.null(sys_info$disk)) {
  cat(sprintf("Disk: %s\n", sys_info$disk))
}

# R Environment
cat(sprintf("R Version: %s\n", R.version.string))
cat(sprintf("R_MAX_VSIZE: %s\n", Sys.getenv("R_MAX_VSIZE", "not set")))

# Pipeline-specific analysis  
cat("\n🏗️ Pipeline Environment\n")
cat("────────────────────\n")

# Check core allocation for pipeline
if (is.numeric(sys_info$cores) && sys_info$cores > 0) {
  cores_per_dataset <- floor(sys_info$cores * 0.25)
  total_pipeline_cores <- cores_per_dataset * 3
  cat(sprintf("Pipeline cores per dataset: %d (25%% of %d)\n", cores_per_dataset, sys_info$cores))
  cat(sprintf("Total pipeline utilization: %d cores (%.1f%%)\n", 
             total_pipeline_cores, (total_pipeline_cores/sys_info$cores)*100))
}

# Check critical directories and files
cat("\n📁 Project Structure\n")
cat("──────────────────\n")

critical_paths <- c(
  "scripts/run_three_datasets.R",
  "scripts/enhanced_pipeline_logger_v2.R", 
  "R/utils/parallel_utils.R",
  "logs/"
)

for (path in critical_paths) {
  if (file.exists(path)) {
    if (dir.exists(path)) {
      files_count <- length(list.files(path))
      cat(sprintf("✅ %s (%d files)\n", path, files_count))
    } else {
      info <- file.info(path)
      cat(sprintf("✅ %s (%.1f KB)\n", path, info$size/1024))
    }
  } else {
    cat(sprintf("❌ %s (missing)\n", path))
  }
}

# Current pipeline status summary
cat(sprintf("\n⏰ Analysis completed at %s\n", format(Sys.time(), "%Y-%m-%d %H:%M:%S")))

# Clean up and flush
flush.console()
gc(verbose = FALSE)